# Merge - spojování tabulek v Pandas

V Pandas je ekvivalentem SQL příkazu `JOIN` funkce `merge`. Slouží ke spojení dvou nebo více tabulek do jedné na základě zadaných podmínek.

Dostupné typy spojení:
- `left` - použije všechny řádky z levé tabulky
- `right` - použije všechny řádky z pravé tabulky
- `inner` - použije pouze společné řádky (výchozí)
- `outer` - použije všechny řádky z obou tabulek
- `cross` - spojí každý řádek s každým (od Pandas 1.2.0)

## 1. Import knihovny a příprava dat

In [ ]:
import pandas as pd

In [ ]:
# Vytvoříme DataFrame s názvy krajů (simulace dat ze souboru)
df_province = pd.DataFrame({
    'province': ['DOLNÍ SLEZSKO', 'LUBUŠSKO', 'ZÁPADNÍ POMOŘANSKO', 
                 'VELKOPOLSKO', 'MALOPOLSKO', 'MAZOVSKO']
})

In [ ]:
df_province

In [ ]:
# Vytvoříme slovník krajů (některé existující + jeden nový, který ještě není v datech)
d_province = pd.DataFrame(
    columns=['d_province'],
    data=['DOLNÍ SLEZSKO', 'LUBUŠSKO', 'ZÁPADNÍ POMOŘANSKO', 'VARŠAVSKÝ OBVOD']
)

In [ ]:
d_province

## 2. Funkce merge - základní parametry

```python
pd.merge(
    left,      # první DataFrame
    right,     # druhý DataFrame
    how,       # typ spojení: 'left', 'right', 'inner', 'outer', 'cross'
    on,        # sloupec pro spojení (pokud má stejný název v obou tabulkách)
    left_on,   # sloupec z levé tabulky
    right_on   # sloupec z pravé tabulky
)
```

Parametry `left_on` a `right_on` používáme, když se názvy sloupců liší.

## 3. LEFT JOIN

Použije všechny řádky z levé tabulky. Pokud není shoda v pravé tabulce, hodnoty budou `NaN`.

In [ ]:
# Spojení pomocí left join
left_join_example = pd.merge(
    left=df_province,
    right=d_province,
    how='left',
    left_on=['province'],
    right_on=['d_province']
)

In [ ]:
left_join_example

**Otázka:** Proč mají některé řádky hodnotu `NaN` ve sloupci `d_province`?

## 4. RIGHT JOIN

Použije všechny řádky z pravé tabulky. Je to opak `left join` - můžeme je vzájemně zaměnit prohozením tabulek.

In [ ]:
# Spojení pomocí right join
right_join_example = pd.merge(
    left=df_province,
    right=d_province,
    how='right',
    left_on=['province'],
    right_on=['d_province']
)

In [ ]:
right_join_example

**Otázka:** Který kraj se objevil, který nebyl v původních datech `df_province`?

## 5. INNER JOIN

Vrátí pouze řádky, které existují v obou tabulkách (průnik). Toto je **výchozí** typ spojení.

In [ ]:
# Spojení pomocí inner join
inner_join_example = pd.merge(
    left=df_province,
    right=d_province,
    how='inner',  # můžeme vynechat, je to výchozí hodnota
    left_on=['province'],
    right_on=['d_province']
)

In [ ]:
inner_join_example

In [ ]:
print(f"Počet řádků: {len(inner_join_example)}")

## 6. OUTER JOIN

Vrátí všechny řádky z obou tabulek (sjednocení). Chybějící hodnoty budou `NaN`.

In [ ]:
# Spojení pomocí outer join
outer_join_example = pd.merge(
    left=df_province,
    right=d_province,
    how='outer',
    left_on=['province'],
    right_on=['d_province']
)

In [ ]:
outer_join_example

In [ ]:
print(f"Počet řádků: {len(outer_join_example)}")

**Otázka:** Proč má `outer join` více řádků než `inner join`?

## 7. CROSS JOIN

Spojí každý řádek z levé tabulky s každým řádkem z pravé tabulky. **Pozor:** Může vytvořit velmi velkou tabulku! (n × m řádků)

In [ ]:
# Spojení pomocí cross join
cross_join_example = pd.merge(
    left=df_province,
    right=d_province,
    how='cross'
    # u cross join nezadáváme podmínku spojení
)

In [ ]:
cross_join_example

In [ ]:
print(f"Počet řádků: {len(cross_join_example)}")
print(f"Očekáváno: {len(df_province)} × {len(d_province)} = {len(df_province) * len(d_province)}")

## 8. Merge s podmínkami

V Pandas můžeme při `merge` použít pouze operátor `==`. Pro složitější podmínky (jako `>=`, `<` atd.) musíme:
1. Nejprve udělat `cross join`
2. Poté filtrovat pomocí `loc` nebo `query`

In [ ]:
# Najdeme kraje, kde název v province je >= než název v d_province
greater_province = pd.merge(
    left=df_province,
    right=d_province,
    how='cross'
)

In [ ]:
# Filtrujeme výsledek
result = greater_province.loc[
    greater_province['province'] >= greater_province['d_province']
].sort_values('province')

In [ ]:
result

---
## Úlohy na opravu chyb

V následujících buňkách jsou chyby. Opravte je.

### Úloha 1: Opravte chybu v merge

In [ ]:
# OPRAVTE: Chceme left join, ale kód nefunguje správně
result = pd.merge(
    left=df_province,
    right=d_province,
    how='left',
    on='province'  # CHYBA: sloupce mají různé názvy
)

### Úloha 2: Opravte syntaxi

In [ ]:
# OPRAVTE: Cross join nefunguje
result = pd.merge(
    left=df_province,
    right=d_province,
    how='cross',
    left_on=['province'],  # CHYBA: u cross join se neuvádí podmínka
    right_on=['d_province']
)

---
## Cvičení

### Cvičení 1: Normalizace datasetu

V listech souboru **dims.xlsx** jsou slovníky pro data ze souboru **product_prices_cleaned.csv**. Pomocí `merge` normalizujte data podle následujících kroků:

1. Načtěte obsah listů souboru **dims.xlsx** do samostatných `DataFrame`. Pro přehlednost pojmenujte rámce podle názvů listů.

2. Načtěte data ze souboru **product_prices_cleaned.csv** do proměnné `df`.

3. Na základě listu **d_province** pomocí sloupce `id` přidejte do rámce `df` sloupec `province_id`.

4. Na základě listu **d_product** přidejte do rámce `df` sloupec `product_id`.

5. Z tabulky extrahujte pouze sloupce, které odkazují na jiné tabulky (např. **product_id**) a sloupce **value**, **date**. Myslíte, že je to přehlednější? Jaké jsou potenciální výhody tohoto přístupu?

> Více o normalizaci databází najdete na [odkazu](https://www.sqlshack.com/what-is-database-normalization-in-sql-server/).

In [ ]:
# Váš kód zde


### Cvičení 2: Merge s podmínkami

Pomocí surových dat ze souboru **product_prices_cleaned.csv** zjistěte, kolikrát byla historicky uvedena nižší cena pro daný produkt, kraj a měsíc. Postupujte takto:

1. Spojte tabulku samu se sebou. Jaký typ spojení byste měli použít?

2. Filtrujte data tak, abyste našli dřívější roky a hodnoty menší než v aktuálním roce v daném kraji.

3. Seskupte data odpovídajícím způsobem.

Který produkt(y) měl(y) nejvíce takových výskytů?

> Všimněte si, co se stane s názvy sloupců, když sloupce nejsou použity jako podmínka spojení, ale mají stejné názvy.

In [ ]:
# Váš kód zde


---
## Přehled použitých metod a funkcí

| Funkce/Metoda | Popis |
|---------------|-------|
| `pd.merge()` | Spojení dvou DataFrame na základě společných sloupců |
| `how='left'` | Left join - všechny řádky z levé tabulky |
| `how='right'` | Right join - všechny řádky z pravé tabulky |
| `how='inner'` | Inner join - pouze společné řádky (výchozí) |
| `how='outer'` | Outer join - všechny řádky z obou tabulek |
| `how='cross'` | Cross join - kartézský součin (každý s každým) |
| `on` | Sloupec pro spojení (stejný název v obou tabulkách) |
| `left_on` | Sloupec z levé tabulky pro spojení |
| `right_on` | Sloupec z pravé tabulky pro spojení |
| `df.loc[]` | Filtrování DataFrame pomocí logické podmínky |
| `df.sort_values()` | Seřazení DataFrame podle sloupce |